In [1]:
import torch, torchvision
from torchvision import transforms
import numpy as np
import cv2
import os
import subprocess
import shutil
import json
import time
import re
import glob
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from detectron2.structures.instances import Instances
from multiprocessing import Process, Manager
import gc
from PIL import Image, ImageDraw, ImageFont


/home/user/miniconda3/envs/int_venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### PATHS
# Do not use directory with negatives
in_dir = "/mnt/nis_lab_research/data/coco_files/og/test/ft_test1_c10"
# in_dir = "/mnt/nis_lab_research/data/coco_files/og/test/ft_test1"
res_out_dir = "./res_out_dir"
vis_out_dir = "./res_out_dir/vis"
# cat_path = "/mnt/nis_lab_research/data/elem_cat/cat_neg_27.json"
cat_path = "/mnt/nis_lab_research/data/elem_cat/cat_neg_10.json"
master_url_dict_path = '/mnt/nis_lab_research/data/top-1m/top-1m-mapping.json'


# OBJECT DETECTOR MODELS
# # 2 OD
od_paths = [
    ("/mnt/nis_lab_research/data/coco_files/agg/class_2_rem/out/far_shah_b1-b5_b8_train_c0/model_final.pth", 0.50), # small
    ("/mnt/nis_lab_research/data/coco_files/agg/class_2_rem/out/far_shah_b1-b5_b8_train_c1/model_final.pth", 0.75)  # large
]

# # 10 OD
# od_paths = [
#     ("/mnt/nis_lab_research/data/coco_files/agg/class_1_rem/out/far_shah_b1-b5_b8_train_c0/model_final.pth", .75),
#     ("/mnt/nis_lab_research/data/coco_files/agg/class_1_rem/out/far_shah_b1-b5_b8_train_c1/model_final.pth", .75),
#     ("/mnt/nis_lab_research/data/coco_files/agg/class_1_rem/out/far_shah_b1-b5_b8_train_c2/model_final.pth", .75),
#     ("/mnt/nis_lab_research/data/coco_files/agg/class_1_rem/out/far_shah_b1-b5_b8_train_c3/model_final.pth", .75),
#     ("/mnt/nis_lab_research/data/coco_files/agg/class_1_rem/out/far_shah_b1-b5_b8_train_c4/model_final.pth", .75),
#     ("/mnt/nis_lab_research/data/coco_files/agg/class_1_rem/out/far_shah_b1-b5_b8_train_c5/model_final.pth", .75),
#     ("/mnt/nis_lab_research/data/coco_files/agg/class_1_rem/out/far_shah_b1-b5_b8_train_c6/model_final.pth", .75),
#     ("/mnt/nis_lab_research/data/coco_files/agg/class_1_rem/out/far_shah_b1-b5_b8_train_c7/model_final.pth", .75),
#     ("/mnt/nis_lab_research/data/coco_files/agg/class_1_rem/out/far_shah_b1-b5_b8_train_c8/model_final.pth", .75),
#     ("/mnt/nis_lab_research/data/coco_files/agg/class_1_rem/out/far_shah_b1-b5_b8_train_c9/model_final.pth", .75)
# ]

# CLASSIFIER MODEL
# # 10 Class Agg + Neg
# class_path = "/mnt/nis_lab_research/data/class_data/pth/far_shah_b1-b5_b8_train_c10_neg/model_final.pth"
class_path = "/mnt/nis_lab_research/data/class_data/pth/full_train_neg_c10/model_final.pth"
# 27 Class + Neg
# class_path = "/mnt/nis_lab_research/data/class_data/pth/far_shah_b1-b5_b8_train_neg/model_final.pth"

# ADJUSTABLES
bg_color = "white"
# Possibly adjust padding to fixed value in the future
padding = 0.15
border = 0
# If True negatively identified cases will be removed before calulating metrics
remove_neg = True
keep_clickable_elems_only = True
iou_thold = .5
neg_class_name = "Random"

In [3]:
# Setting CUDA devices as visible
cuda_devices = "2,3"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_devices

In [4]:
available_devices = [torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())]
print("Available CUDA devices:")
for i, device_name in enumerate(available_devices):
    print(f"  {i}: {device_name}")

Available CUDA devices:
  0: Tesla P100-PCIE-16GB
  1: Tesla P100-PCIE-16GB


In [5]:
torch.cuda.empty_cache()
gc.collect()

0

In [6]:
def get_soi(str1, start_char, end_char):
    str1 = str(str1)
    offst = len(start_char)
    ind1 = str1.find(start_char)
    ind2 = str1.find(end_char)
    s_str = str1[ind1+offst:ind2]
    return s_str

In [7]:
def convert_bbox_xywh(b):
    x1, y1, x2, y2 = b
    x = x1
    y = y1
    w = x2 - x1
    h = y2 - y1
    return [x, y, w, h]

In [8]:

def createDataDict (fn, outputs):
    img_shape = list(outputs["instances"].image_size)
    img_h = int(img_shape[0])
    img_w = int(img_shape[1])
    ann_list = []

    class_list = get_soi(outputs["instances"].pred_classes, "[", "]").split(",")
    
    if class_list[0] != "":

        class_list_new = []
        for each in class_list:
            if each.strip().isdigit():
                class_list_new.append(int(each.strip()))
            else:
                print(f"Invalid class ID: {each}")

        bbox_list = get_soi(outputs["instances"].pred_boxes, "[[", "]]").split("]")
        bbox_list_new = []
        for each in bbox_list:
            bbox = re.sub("['[,\n]", "", each).split(" ")
            bbox_new = []
            for item in bbox:
                if item != "":
                    bbox_new.append(float(item))
            bbox_new = convert_bbox_xywh(bbox_new)
            bbox_list_new.append(bbox_new)

        for i in range(0, len(class_list)):
            # og was "bbox_mode": "<BoxMode.XYWH_ABS: 1>"
            ann_list.append({"iscrowd": 0, "bbox": bbox_list_new[i], "category_id": class_list_new[i], "bbox_mode": 0})
    
    data_dict = {
        "file_name": fn,
        "height": img_h,
        "width": img_w, 
        "annotations": ann_list
    }
 
    return data_dict

In [9]:
def crop_image(file_path, bounding_box, padding):
    
    with Image.open(file_path) as img:
        
        x_min, y_min, width, height = bounding_box

        # Calculate padding in pixels
        pad_width = int(width * padding)
        pad_height = int(height * padding)

        # Adjust the bounding box with padding
        x_min = max(x_min - pad_width, 0)
        y_min = max(y_min - pad_height, 0)
        x1 = min(x_min + width + 2 * pad_width, img.width)
        y1 = min(y_min + height + 2 * pad_height, img.height)
        
        cropped_img = img.crop((x_min, y_min, x1, y1))
        
        return cropped_img

In [10]:
def paste_to_bg(image, background_color, bg_width, bg_height):
    
    # Create a new image with the specified background color and dimensions
    background = Image.new('RGB', (bg_width, bg_height), background_color)

    # Calculate the position to paste the image so it's centered
    x = (bg_width - image.width) // 2
    y = (bg_height - image.height) // 2

    # Paste the image onto the background
    background.paste(image, (x, y), image if image.mode == 'RGBA' else None)

    return background

In [11]:
def resize_ar_lock(img, target_size):

    original_width, original_height = img.size
    target_width, target_height = target_size

    # Calculate scaling factor
    scaling_factor = min(target_width / original_width, target_height / original_height)

    # Calculate new dimensions
    new_width = max(int(original_width * scaling_factor), 1)
    new_height = max(int(original_height * scaling_factor), 1)

    # Resize the image
    resized_img = img.resize((new_width, new_height))

    return resized_img

In [12]:
def gen_rand_str(length):
    characters = string.ascii_letters + string.digits
    random_string = ''.join(random.choice(characters) for i in range(length))
    return random_string

In [13]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [14]:
def process_image(img_fp, bbox_list, padding, bg_color, border):
    
    # Create an empty list to store processed images
    processed_images = []

    for j, bbox in enumerate(bbox_list):

        try:
            elem_img = crop_image(img_fp, bbox, padding)
            e_w = elem_img.size[0]
            e_h = elem_img.size[1]

            if e_w < e_h:
                elem_img = paste_to_bg(elem_img, bg_color, e_h + border, e_h + border)
            elif e_w > e_h:
                elem_img = paste_to_bg(elem_img, bg_color, e_w + border, e_w + border)
                
            # elem_img = transform(elem_img)
            processed_images.append(elem_img)

        except Exception as e:
            print(img_fp)
            print(e)

    # Return the list of processed images
    return processed_images


In [15]:
def draw_bounding_boxes(image_path, bbox_list, label_list, output_path, color = 'red', thickness=2):
    # Open the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Load a font
    font = ImageFont.load_default()

    # Draw bounding boxes and labels
    for bbox, label in zip(bbox_list, label_list):
        x, y, w, h = bbox
        draw.rectangle([x, y, x+w, y+h], outline=color, width = 2)
        draw.text((x, y), label, fill=color, font=font)

    # Save the new image
    image.save(output_path)

In [16]:
def draw_bounding_boxes_in_mem(image, bbox_list, label_list, color='red', thickness=2):
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    for bbox, label in zip(bbox_list, label_list):
        x, y, w, h = bbox
        draw.rectangle([x, y, x + w, y + h], outline=color, width=thickness)
        draw.text((x, y), label, fill=color, font=font)

    return image

In [17]:
def merge_outputs(outputs):
    l = []
    for out in outputs:
        l.append(out["instances"])
    new = Instances.cat(l)
    return {"instances": new}

In [18]:
def keep_all_but_first_part(s):
    parts = s.split('-')
    if len(parts) > 1:
        return '-'.join(parts[1:])
    return s

In [19]:
if os.path.exists(res_out_dir):
    shutil.rmtree(res_out_dir)
os.makedirs(vis_out_dir)

In [20]:
img_path_list = [os.path.join(in_dir, "images", img_path) for img_path in sorted(os.listdir(os.path.join(in_dir, "images")))]

In [21]:
obj_det_pred_list = []

for od in od_paths:
    
    setup_logger()
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
    # cfg.MODEL.WEIGHTS = os.path.join("/home/dtron2_user/ls_dtron2_full/model/output", "model_final.pth")
    cfg.MODEL.WEIGHTS = od[0]
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = od[1] 
    obj_det_pred = DefaultPredictor(cfg)
    obj_det_pred_list.append(obj_det_pred)

[06/28 03:38:01 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /mnt/nis_lab_research/data/coco_files/agg/class_2_rem/out/far_shah_b1-b5_b8_train_c0/model_final.pth ...
[06/28 03:38:04 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /mnt/nis_lab_research/data/coco_files/agg/class_2_rem/out/far_shah_b1-b5_b8_train_c1/model_final.pth ...


In [22]:
classifier = torch.load(class_path)
classifier.eval()

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [23]:
with open(cat_path, 'r') as f:
    cats = json.load(f)

In [24]:
neg_class_id = ""

for key, value in cats.items():
    if value == neg_class_name:
        neg_class_id = key


In [25]:
neg_class_id

'8'

In [26]:
with open(master_url_dict_path) as f:
    master_url_dict = json.load(f)

In [27]:
def get_clickable_elements(domains):

    obj_list = []

    for dom in domains:

        print(dom)

        url = f'https://{dom}'

        cmd = ['node', '../../get_clickable_elems/get_clickable_elems.js', url]
        try:
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)
            print(result.stdout)  # Print the standard output from the command
        except subprocess.CalledProcessError as e:
            print(f"Error executing command: {e}")
            print(f"Command output: {e.output}")

        time.sleep(1)

        json_path = os.path.join('./data', f'{dom}.json' )
        if os.path.exists(json_path):
            with open(json_path) as f:
                jobj = json.load(f)

            obj_list.append(jobj)

            return obj_list

        return None


In [28]:
def calculate_iou(boxA, boxB):
    # Convert from [x, y, w, h] to [x1, y1, x2, y2]
    x1A, y1A, x2A, y2A = boxA[0], boxA[1], boxA[0] + boxA[2], boxA[1] + boxA[3]
    x1B, y1B, x2B, y2B = boxB[0], boxB[1], boxB[0] + boxB[2], boxB[1] + boxB[3]
    
    # Determine the coordinates of the intersection rectangle
    xA = max(x1A, x1B)
    yA = max(y1A, y1B)
    xB = min(x2A, x2B)
    yB = min(y2A, y2B)
    
    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    # Compute the area of both the prediction and true bounding boxes
    boxAArea = (x2A - x1A) * (y2A - y1A)
    boxBArea = (x2B - x1B) * (y2B - y1B)
    
    # Compute the area of union
    unionArea = boxAArea + boxBArea - interArea
    
    # Compute the Intersection over Union by dividing the intersection area by the union area
    iou = interArea / unionArea
    
    return iou

In [29]:
def get_non_clickable_elem_ind_list(img_path, pred_bbox_list):
    # Assuming the existence of keep_all_but_first_part, master_url_dict, get_clickable_elements, and calculate_iou
    
    # Extract the key from the image path
    key = keep_all_but_first_part(img_path.split('/')[-1])[:-7]
    
    # Get the domain from the master_url_dict using the key
    domain = master_url_dict.get(key)
    if not domain:
        raise ValueError(f"Key {key} not found in master_url_dict")

    # Get clickable elements for the domain
    ces = get_clickable_elements([domain])

    if ces == None:
        return None, None
    else:
        ces_obj = ces[0]


        # Build list of clickable bounding boxes
        clickable_bbox_list = [[ce['x'], ce['y'], ce['width'], ce['height']] for ce in ces_obj]

        # List to hold non-clickable element indices
        non_clickable_ind_list = []

        # Check each predicted bounding box against clickable bounding boxes
        for i, p_bbox in enumerate(pred_bbox_list):
            is_non_clickable = True
            for ces in ces_obj:
                ces_bbox = [ces['x'], ces['y'], ces['width'], ces['height']]
                iou_score = calculate_iou(p_bbox, ces_bbox)
                if iou_score >= iou_thold:
                    is_non_clickable = False
                    break
            if is_non_clickable:
                non_clickable_ind_list.append(i)

        return non_clickable_ind_list, clickable_bbox_list



In [30]:
master_det_dict = []

for i, img_path in enumerate(img_path_list[0:25]):

    vis_outpath = os.path.join(vis_out_dir, os.path.basename(img_path))

    # Creating master dictionary of detected elements
    img = cv2.imread(img_path)
    
    pred_out_list = []
    for od_pred in obj_det_pred_list:
        pred_out_list.append(od_pred(img))
    outputs = merge_outputs(pred_out_list)
    
    print('###############################################################################################')
    print(i, img_path)
    
    data_dict = createDataDict(img_path, outputs)
    pred_bbox_list = [ann["bbox"] for ann in data_dict["annotations"]]

    print("predicted number:", len(data_dict["annotations"]))

    if keep_clickable_elems_only:
        nce_list, ce_bbox_list = get_non_clickable_elem_ind_list(img_path, pred_bbox_list)

        if nce_list == None and ce_bbox_list == None:
            pass
        
        else:
            print('non clickable', len(nce_list))
            print('clickable', len(ce_bbox_list))

            z_list = ['0' for _ in range(len(ce_bbox_list))]
            
            draw_bounding_boxes(img_path, ce_bbox_list, z_list, vis_outpath, color='green')

            for ind in sorted(nce_list, reverse=True):
                data_dict["annotations"].pop(ind)
                pred_bbox_list.pop(ind)

            print("cleaned number after removing non clickable elements:", len(data_dict["annotations"]))

    elem_img_list = process_image(data_dict["file_name"], pred_bbox_list, padding, bg_color, border)
    
    pred_ids = []
    pred_classes = []
    remove_list = []
    
    for j, img in enumerate(elem_img_list):
        
        img_t = transform(img.convert('RGB')).unsqueeze(0).to('cuda')
        
        with torch.no_grad():
            output = classifier(img_t)
        _, predicted = torch.max(output, 1)
        # OG
        # pred_class_id = str(predicted.item() + 1)
        pred_class_id = str(predicted.item())
        pred_class_name = cats[pred_class_id]
        
        if remove_neg and pred_class_id == neg_class_id:
            remove_list.append(j)

        pred_ids.append(pred_class_id)
        pred_classes.append(pred_classes)
        # data_dict["annotations"][j]["category_id"] = int(pred_class_id)
        data_dict["annotations"][j]["category_id"] = pred_class_name
    
    if remove_neg and remove_list:
         for ind in sorted(remove_list, reverse=True):
            data_dict["annotations"].pop(ind)
            pred_bbox_list.pop(ind)
            pred_ids.pop(ind)
            pred_classes.pop(ind)
            
    print("cleaned number after removing negative classes:", len(data_dict["annotations"]))
    
    if os.path.exists(vis_outpath):
        draw_bounding_boxes(vis_outpath, pred_bbox_list, pred_ids, vis_outpath, color='red')
    else:
        draw_bounding_boxes(img_path, pred_bbox_list, pred_ids, vis_outpath, color='red')
    
    master_det_dict.append(data_dict)
    
res_outpath = os.path.join(res_out_dir, "results.json")
print("writing out results to", res_outpath)
with open(res_outpath, 'w+') as f:
    json.dump(master_det_dict, f, indent=4)

/home/user/miniconda3/envs/int_venv/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


0 /mnt/nis_lab_research/data/coco_files/og/test/ft_test1_c10/images/0UnHYuFgKVpkj5fl-climbkalymnos_ss.png
predicted number: 28
climbkalymnos.com
2024-06-28 03:38:07 [info]: Attempt 1 to process https://climbkalymnos.com
2024-06-28 03:38:08 [info]: Navigated to https://climbkalymnos.com
2024-06-28 03:38:24 [info]: Successfully processed https://climbkalymnos.com

non clickable 20
clickable 23
cleaned number after removing non clickable elements: 8
cleaned number after removing negative classes: 7
1 /mnt/nis_lab_research/data/coco_files/og/test/ft_test1_c10/images/0XQJNLHix7U3ywoz-trollalley_ss.png
predicted number: 43
trollalley.com
2024-06-28 03:38:29 [info]: Attempt 1 to process https://trollalley.com
2024-06-28 03:38:30 [info]: Navigated to https://trollalley.com
2024-06-28 03:38:39 [info]: Successfully processed https://trollalley.com

non clickable 37
clickable 95
cleaned number after removing non clickable elements: 6
cleaned number after removing negative classes: 6
2 /mnt/nis_la

KeyboardInterrupt: 